# Create 5-degree monthly average gridded dataset for SIF and XCO2
---

In [1]:
import sys
sys.path.insert(0, "../src")

import pandas as pd 
import xarray

import data_utils

In [2]:
DS_sif = xarray.open_dataset("../data/exp_pro/OCO2_Lite_SIF740.nc")
DS_xco2 = xarray.open_dataset("../data/exp_pro/OCO2_Lite_XCO2_land.nc")

In [6]:
DS_sif.lon.min()

<xarray.DataArray 'lon' ()>
array(-180., dtype=float32)

In [5]:
df_sif_monthly = (
    data_utils.regrid(DS_sif, res=5)
    .groupby(["lon", "lat"])
    .resample("1MS")
    .mean()
    .drop(columns=["lon", "lat"])
    .reset_index()
)
df_xco2_monthly = (
    data_utils.regrid(DS_xco2, res=5)
    .groupby(["lon", "lat"])
    .resample("1MS")
    .mean()
    .drop(columns=["lon", "lat"])
    .reset_index()
)

# Merge dataframes
DS_grid = (
    pd.merge(df_sif_monthly, df_xco2_monthly, on=["lon", "lat", "time"], how="outer")
    .set_index(["lon", "lat", "time"])
    .to_xarray()
)
DS_grid

<xarray.Dataset>
Dimensions:   (lat: 35, lon: 72, time: 75)
Coordinates:
  * lon       (lon) float64 -177.5 -172.5 -167.5 -162.5 ... 167.5 172.5 177.5
  * lat       (lat) float64 -87.5 -82.5 -77.5 -72.5 ... 67.5 72.5 77.5 82.5
  * time      (time) datetime64[ns] 2014-09-01 2014-10-01 ... 2020-11-01
Data variables:
    sif       (lon, lat, time) float32 nan nan 0.013318249 ... nan nan nan
    sif_var   (lon, lat, time) float32 nan nan 0.2780175 ... nan nan nan
    xco2      (lon, lat, time) float32 nan nan nan nan nan ... nan nan nan nan
    xco2_var  (lon, lat, time) float32 nan nan nan nan nan ... nan nan nan nan

In [11]:
DS_grid.to_netcdf("../data/exp_pro/OCO2_5deg_monthly.nc", format="NETCDF4")
DS_grid.close()